In [2]:
using GLMakie, Observables
GLMakie.activate!()

In [34]:
# Create Figure with one axis
fig = Figure()
ax = Axis(fig[1,1], title="Interactive Projectile")

# Set Plot Limits
xlims!(ax, 0, 500)
ylims!(ax, 0, 500)

# set inital slider values
init_power = 50
init_angle = 45

# Figure Interactibles
power_slider = Slider(fig[2,1], range= 30:100, startvalue= init_power)
angle_slider = Slider(fig[1,2], range= 10:80, startvalue= init_angle, horizontal= false)
fire_button = Button(fig[2,2], label="Fire")

angle_label = lift(a -> string("angle = ", a), angle_slider.value)
power_label = lift(p -> string("power = ", p), power_slider.value)

Label(fig[3,1][1,1], power_label, tellwidth = false)
Label(fig[3,1][1,2], angle_label, tellwidth = false)

# Projectile statistics
max_height = Observable(0.0)
max_dist = Observable(0.0)
travel = Observable(0.0)

mh_label = lift(h -> string("Max Height: ", round(h; digits=3), "m"), max_height)
md_label = lift(d -> string("Travel Distance: ", round(d; digits=3), "m"), max_dist)
tr_label = lift(t -> string("Travel Time: ", round(t; digits=3), "s"), travel)

Label(fig[4,1][1,1], mh_label, tellwidth = false)
Label(fig[4,1][1,2], md_label, tellwidth = false)
Label(fig[4,1][1,3], tr_label, tellwidth = false)

# Arrow Observable
arrow = lift(angle_slider.value, power_slider.value) do θ, R
    Δx = R*cosd(θ)
    Δy = R*sind(θ)
    return (Δx, Δy)
end

# Plot inital arrow
arrows!([0], [0], [init_power*cosd(init_angle)], [init_power*sind(init_angle)])

# Update arrow on slider changes.
on(arrow) do (dx, dy)
    empty!(ax)
    arrows!([0], [0], [dx], [dy])
end

# Get projectiles position at time t
function getPoint(t)
    V₀ = power_slider.value[]
    Θ = angle_slider.value[]
    g = 9.8
    x = V₀ * cosd(Θ)t
    y = V₀ * sind(Θ)t - 0.5g * t^2
    return Point2f(x, y)
end

# Start Projectile on button click.
on(fire_button.clicks) do n
    empty!(ax)
    points = Observable([Point2f(0,0)])
    max_height[] = 0
    max_dist[] = 0
    scatter!(ax, points)
    t = 0
    while true
        newP = getPoint(t)
        if newP[2] < 0
            travel[] = t
            break
        end
        points[] = push!(points[], newP)
        if newP[2] > max_height[]
            max_height[] = newP[2]
        end
        max_dist[] = newP[1]
        t += 0.1
    end   
end

display(fig)

GLMakie.Screen(...)